In [1]:
import numpy as np
import pandas as pd

import json
import glob
import os
import re

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
base = "../exp/"
paths = glob.glob(base + "rand/*.json") + glob.glob(base + "rand_live/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    **algo,
    'potential': run['potential'],
    'graph': run['args'][1],
    'metric': 'predicted' if run['program'] in ['predicted_queries', 'predicted_queries_no_topo'] else run['args'][3],
    'topo': 'no_topo' not in run['program']
} for run in data for algo in run['algo_runs']])

queries['num_queue_pops_k'] = queries['num_queue_pops'] / 1000
queries['increase_percent'] = (queries['result'] / queries['lower_bound'] - 1) * 100

In [4]:
runtime_pattern = re.compile(".*running time : (\\d+)musec.*")

def parse_flowcutter_partition_output(path):
    stats = { 'cch_ordering_running_time_s': 0.0 }

    with open(path, 'r') as f:
        for line in f:
            if not 'graph' in stats:
                stats['graph'] = line.strip()
            else:
                match = runtime_pattern.match(line)
                if match:
                    stats['cch_ordering_running_time_s'] += int(match[1]) / 1000000

    return stats

In [5]:
cch_ordering = pd.DataFrame.from_records([parse_flowcutter_partition_output(path) for path in glob.glob(base + "preprocessing/*.out")])

In [6]:
data = [json.load(open(path)) for path in glob.glob(base + "preprocessing/*.json")]
prepros = pd.DataFrame.from_records([{
    **run,
    'potential': { 'cchpot_pre': 'lower_bound_cch_pot', 'multi_metric_pre': 'multi_metric_pot', 'interval_min_pre': 'interval_min_pot' }[run['program']],
    'graph': run['args'][1],
} for run in data])

In [7]:
data = [json.load(open(path)) for path in glob.glob(base + "customization/*.json")]
customs = pd.DataFrame.from_records([{
    **run,
    'potential': { 'multi_metric_live_customization': 'multi_metric_pot', 'interval_min_live_customization': 'interval_min_pot' }[run['program']],
    'graph': run['args'][1],
    'metric': run['args'][3],
} for run in data])

In [8]:
queries.query('topo').groupby(['graph', 'metric', 'potential']).mean()[['running_time_ms', 'num_queue_pops_k', 'increase_percent']]

running_time_ms  \
graph                                              metric          potential                              
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot               37.69   
                                                                   lower_bound_cch_pot           127.98   
                                                                   multi_metric_pot              119.00   
                                                                   zero                         1511.53   
                                                   lite_live_data  interval_min_pot               29.12   
                                                                   lower_bound_cch_pot           236.54   
                                                                   multi_metric_pot              170.05   
                                                                   zero                         1545.01   
                                                   predicted       interval_min_pot               22.17   
                                                                   lower_bound_cch_pot           137.53   
                                                                   multi_metric_pot              117.68   
                                                                   zero                         1473.06   
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot               25.44   
                                                                   lower_bound_cch_pot           152.22   
                                                                   multi_metric_pot              131.23   
                                                                   zero                         2732.57   
                                                   predicted       interval_min_pot               11.49   
                                                                   lower_bound_cch_pot           102.58   
                                                                   multi_metric_pot               95.30   
                                                                   zero                         2696.20   

                                                                                        num_queue_pops_k  \
graph                                              metric          potential                               
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot                11.31   
                                                                   lower_bound_cch_pot             89.65   
                                                                   multi_metric_pot                79.50   
                                                                   zero                          1671.60   
                                                   lite_live_data  interval_min_pot                 7.55   
                                                                   lower_bound_cch_pot            158.26   
                                                                   multi_metric_pot               110.04   
                                                                   zero                          1672.51   
                                                   predicted       interval_min_pot                 5.12   
                                                                   lower_bound_cch_pot             92.32   
                                                                   multi_metric_pot                74.62   
                                                                   zero                          1672.64   
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot                 7.44   
                                                                   lower_bound_cch_pot            102.23   
     

In [9]:
queries.query('~topo').groupby(['graph', 'metric', 'potential']).mean()[['running_time_ms', 'num_queue_pops_k']]

running_time_ms  \
graph                                              metric          potential                              
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot               65.89   
                                                                   lower_bound_cch_pot           285.99   
                                                                   multi_metric_pot              263.20   
                                                                   zero                         3450.10   
                                                   lite_live_data  interval_min_pot               48.97   
                                                                   lower_bound_cch_pot           545.79   
                                                                   multi_metric_pot              386.72   
                                                                   zero                         3472.14   
                                                   predicted       interval_min_pot               34.50   
                                                                   lower_bound_cch_pot           311.09   
                                                                   multi_metric_pot              262.62   
                                                                   zero                         3416.26   
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot               38.96   
                                                                   lower_bound_cch_pot           325.30   
                                                                   multi_metric_pot              280.99   
                                                                   zero                         5989.37   
                                                   predicted       interval_min_pot               14.19   
                                                                   lower_bound_cch_pot           217.47   
                                                                   multi_metric_pot              200.03   
                                                                   zero                         5952.15   

                                                                                        num_queue_pops_k  
graph                                              metric          potential                              
/home/i11/zeitz/projects/tdpot/data/osm_ger/       heavy_live_data interval_min_pot                71.84  
                                                                   lower_bound_cch_pot            485.56  
                                                                   multi_metric_pot               432.81  
                                                                   zero                          8075.76  
                                                   lite_live_data  interval_min_pot                50.65  
                                                                   lower_bound_cch_pot            825.15  
                                                                   multi_metric_pot               581.33  
                                                                   zero                          8075.22  
                                                   predicted       interval_min_pot                36.05  
                                                                   lower_bound_cch_pot            490.42  
                                                                   multi_metric_pot               400.75  
                                                                   zero                          8079.14  
/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/ live_data       interval_min_pot                38.57  
                                                                   lower_bound_cch_pot            504.32  
                     

In [10]:
baseline = queries.query('~topo').groupby(['potential', 'graph', 'metric']).mean()['running_time_ms']['zero']

In [15]:
table = queries.query('topo') \
    .groupby(['graph', 'metric', 'potential']) \
    .mean()[['running_time_ms', 'num_queue_pops_k', 'increase_percent']] \
    .reset_index(level=2) \
    .merge(baseline, how='left', suffixes=['', '_baseline'], left_index=True, right_index=True) \
    .reset_index() \
    .set_index(['potential', 'graph', 'metric']) \
    .sort_index() \
    .loc[['lower_bound_cch_pot', 'multi_metric_pot', 'interval_min_pot']]

table['speedup'] = table['running_time_ms_baseline'] / table['running_time_ms']


pre_merged = prepros[['graph', 'potential', 'preprocessing', 'build']].fillna(0.0).groupby(['graph', 'potential']).mean()[['preprocessing', 'build']].reset_index() \
    .merge(cch_ordering.groupby('graph').mean()['cch_ordering_running_time_s'], how='left', on='graph')
pre_merged['total_preprocessing_time_s'] = pre_merged['cch_ordering_running_time_s'] + pre_merged['preprocessing'] / 1000 + pre_merged['build'] / 1000
table = table.reset_index().merge(pre_merged[['graph', 'potential', 'total_preprocessing_time_s']], how='left', on=['graph', 'potential'])
table = table.merge(customs.groupby(['graph', 'metric', 'potential']).mean()[['customization']] / 1000, how='left', on=['graph', 'metric', 'potential'])

table['graph'] = table['graph'].map({ '/home/i11/zeitz/projects/tdpot/data/osm_ger/': 'OSM Ger', '/home/i11/zeitz/projects/tdpot/data/ptv20-eur-car/': 'PTV Eur' })

table = table.set_index(['potential', 'graph', 'metric'])
table = table.reindex(index=['predicted', 'live_data', 'lite_live_data', 'heavy_live_data'], level=2)

table = table[['running_time_ms', 'num_queue_pops_k', 'increase_percent', 'speedup', 'total_preprocessing_time_s', 'customization']]
print(table.to_latex(float_format="%.1f"))
table

\begin{tabular}{lllrrrrrr}
\toprule
                 &         &           &  running\_time\_ms &  num\_queue\_pops\_k &  increase\_percent &  speedup &  total\_preprocessing\_time\_s &  customization \\
potential & graph & metric &                  &                   &                   &          &                             &                \\
\midrule
lower\_bound\_cch\_pot & OSM Ger & predicted &            137.5 &              92.3 &              12.2 &     24.8 &                       165.2 &            NaN \\
                 &         & lite\_live\_data &            236.5 &             158.3 &              18.9 &     14.7 &                       165.2 &            NaN \\
                 &         & heavy\_live\_data &            128.0 &              89.6 &              19.1 &     27.0 &                       165.2 &            NaN \\
                 & PTV Eur & predicted &            102.6 &              65.2 &               4.2 &     58.0 &                       249.7 &  

/tmp/ipykernel_6015/3591039987.py:26: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table.to_latex(float_format="%.1f"))


running_time_ms  \
potential           graph   metric                             
lower_bound_cch_pot OSM Ger predicted                 137.53   
                            lite_live_data            236.54   
                            heavy_live_data           127.98   
                    PTV Eur predicted                 102.58   
                            live_data                 152.22   
multi_metric_pot    OSM Ger predicted                 117.68   
                            lite_live_data            170.05   
                            heavy_live_data           119.00   
                    PTV Eur predicted                  95.30   
                            live_data                 131.23   
interval_min_pot    OSM Ger predicted                  22.17   
                            lite_live_data             29.12   
                            heavy_live_data            37.69   
                    PTV Eur predicted                  11.49   
                            live_data                  25.44   

                                             num_queue_pops_k  \
potential           graph   metric                              
lower_bound_cch_pot OSM Ger predicted                   92.32   
                            lite_live_data             158.26   
                            heavy_live_data             89.65   
                    PTV Eur predicted                   65.19   
                            live_data                  102.23   
multi_metric_pot    OSM Ger predicted                   74.62   
                            lite_live_data             110.04   
                            heavy_live_data             79.50   
                    PTV Eur predicted                   58.61   
                            live_data                   84.48   
interval_min_pot    OSM Ger predicted                    5.12   
                            lite_live_data               7.55   
                            heavy_live_data             11.31   
                    PTV Eur predicted                    1.76   
                            live_data                    7.44   

                                             increase_percent  speedup  \
potential           graph   metric                                       
lower_bound_cch_pot OSM Ger predicted                   12.20    24.84   
                            lite_live_data              18.93    14.68   
                            heavy_live_data             19.07    26.96   
                    PTV Eur predicted                    4.16    58.03   
                            live_data                    8.42    39.35   
multi_metric_pot    OSM Ger predicted                    9.94    29.03   
                            lite_live_data              12.97    20.42   
                            heavy_live_data             15.83    28.99   
                    PTV Eur predicted                    3.48    62.46   
                            live_data                    5.85    45.64   
interval_min_pot    OSM Ger predicted                    1.76   154.09   
                            lite_live_data               2.60   119.24   
                            heavy_live_data              4.18    91.53   
                    PTV Eur predicted                    0.36   517.96   
                            live_data                    1.69   235.46   

                                             total_preprocessing_time_s  \
potential           graph   metric                                        
lower_bound_cch_pot OSM Ger predicted                            165.24   
                            lite_live_data                       165.24   
                            heavy_live_data                      165.24   
                    PTV Eur predicted                            249.71   
                            live_data                            249.71   
multi_metric_pot    OSM Ger predicted                            3